In [1]:
import csv
import requests
import os
import pandas as pd
import json
key = os.environ.get("ALPHA_VANTAGE_KEY")


In [9]:
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
CSV_URL = 'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={}'.format(key)

with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    #makeinto pandas dataframe, first row as header
    df = pd.DataFrame(cr)
    headers = df.iloc[0]
    df.columns = [headers]
    df.drop(index=0, axis=0, inplace=True)
    #create list of items as such { value: symbol, label: Name (Symbol) }
    items = []
    for index, row in df.iterrows():
        items.append({'value': row['name'] , 'label': row['symbol'] })
    #save list to json file
    with open('data.json', 'w') as outfile:
        json.dump(items, outfile)
    

In [5]:
from fuzzywuzzy import fuzz

# Parse datajson to fuzzy match similarity with word and return the top 20 matches sorted by closeness match on value and word
def fuzzy(word):
    with open('data.json', 'r') as json_file:
        data = json.load(json_file)
        
        #by first letter of word and label
        dataload = list(filter(lambda x: x['label'].startswith(word[0].upper()), data))

        # Define a function to calculate the similarity score
        def similarity_score(item):
            # Modify this scoring as needed
            value_similarity = fuzz.token_sort_ratio(word.lower(), item['value'].lower())
            label_similarity = fuzz.token_sort_ratio(word.lower(), item['label'].lower())
            return value_similarity + label_similarity

        # Sort the data based on similarity score in descending order
        sorted_data = sorted(dataload, key=similarity_score, reverse=True)

        # Return the top 20 matches
        return sorted_data[:20]
    

fuzzy('tes')


[{'value': 'Tesla Inc', 'label': 'TSLA | Tesla Inc'},
 {'value': 'Telus Corp', 'label': 'TU | Telus Corp'},
 {'value': 'Telos Corp', 'label': 'TLS | Telos Corp'},
 {'value': 'Tenaris S.A.', 'label': 'TS | Tenaris S.A.'},
 {'value': 'Team Inc', 'label': 'TISI | Team Inc'},
 {'value': 'Tapestry Inc', 'label': 'TPR | Tapestry Inc'},
 {'value': 'TSR Inc', 'label': 'TSRI | TSR Inc'},
 {'value': 'Tecnoglass Inc', 'label': 'TGLS | Tecnoglass Inc'},
 {'value': 'Trinseo PLC', 'label': 'TSE | Trinseo PLC'},
 {'value': 'Telesis Bio Inc', 'label': 'TBIO | Telesis Bio Inc'},
 {'value': 'Taseko Mines Ltd', 'label': 'TGB | Taseko Mines Ltd'},
 {'value': 'TXO Partners L.P.', 'label': 'TXO | TXO Partners L.P.'},
 {'value': 'TEGNA Inc', 'label': 'TGNA | TEGNA Inc'},
 {'value': 'Travelzoo', 'label': 'TZOO | Travelzoo'},
 {'value': 'Invesco Solar ETF', 'label': 'TAN | Invesco Solar ETF'},
 {'value': 'Terex Corp', 'label': 'TEX | Terex Corp'},
 {'value': 'Toll Brothers Inc', 'label': 'TOL | Toll Brothers I